In [1]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None


def printm(s): return display(Markdown(s))

/home/prasoon/anaconda3/envs/edusense/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [37]:
pd.__version__

'1.3.5'

# Figure out how many sessions are there, and how many frames for each session

In [2]:
track_analysis_meta_cache = '../cache/analysis_tracking/meta_info'
os.makedirs(track_analysis_meta_cache,exist_ok=True)

In [3]:
base_dir = '/mnt/ci-nas-cache/edulyzeV2/track/'
frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
        
        

In [4]:
for course in frame_file_data:
    print(course)
    print(frame_file_data[course].keys())

17214A
dict_keys(['classinsight-cmu_17214A_dh_2105_201901300930-front', 'classinsight-cmu_17214A_dh_2105_201902130930-front', 'classinsight-cmu_17214A_dh_2105_201902200930-front', 'classinsight-cmu_17214A_dh_2105_201902270930-front', 'classinsight-cmu_17214A_dh_2105_201903060930-front', 'classinsight-cmu_17214A_dh_2105_201903130930-front', 'classinsight-cmu_17214A_dh_2105_201903200930-front', 'classinsight-cmu_17214A_dh_2105_201904030930-front', 'classinsight-cmu_17214A_dh_2105_201904100930-front', 'classinsight-cmu_17214A_dh_2105_201904170930-front', 'classinsight-cmu_17214A_dh_2105_201904240930-front', 'classinsight-cmu_17214A_dh_2105_201905010930-front'])
17214B
dict_keys(['classinsight-cmu_17214B_ph_a21_201901301030-front', 'classinsight-cmu_17214B_ph_a21_201902271030-front', 'classinsight-cmu_17214B_ph_a21_201903061030-front', 'classinsight-cmu_17214B_ph_a21_201903131030-front', 'classinsight-cmu_17214B_ph_a21_201903201030-front', 'classinsight-cmu_17214B_ph_a21_201904031030-front

In [5]:
# course_name, session_name, is_completed, min_frame, max_frame_number, num_frames, session_duration_mins, 
# Next step: Get how many sessions are completed, and 
session_infos = []
for course_name in frame_file_data:
    for session_name in frame_file_data[course_name]:
        # print(session_name)
        session_meta_data = {'course':course_name}
        session_meta_data['session']=session_name.split("-front")[0]
        print(session_meta_data)
        try:
            session_meta_data['start_time'] = pd.to_datetime(session_meta_data['session'].split("_")[-1], format="%Y%m%d%H%M")
        except:
            session_meta_data['start_time'] = pd.to_datetime(session_meta_data['session'].split("_")[-1], format="%Y%m%d%H%M%S")
        session_meta_data['start_hour'] = session_meta_data['start_time'].strftime('%H:%M')
        session_meta_data['classroom'] = "_".join(session_meta_data['session'].split("_")[-3:-1])
        meta_data = frame_file_data[course_name][session_name]
        session_meta_data['completed'] = meta_data['is_completed']
        session_meta_data['num_frames'] = len(meta_data['frame_ids'])
        if len(meta_data['frame_ids']) > 0:
            session_meta_data['max_frame'] = max(meta_data['frame_ids'])
            session_meta_data['duration_in_mins'] = max(meta_data['frame_ids'])//(15*60)
        session_infos.append(session_meta_data)
        # print(course_name, session_name)
df_session_info = pd.DataFrame(session_infos)   

{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201901300930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201902130930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201902200930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201902270930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201903060930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201903130930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201903200930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201904030930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201904100930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201904170930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201904240930'}
{'course': '17214A', 'session': 'classinsight-cmu_17214A_dh_2105_201905010930'}
{'course': '17214B', 'session': 'classin

In [6]:
session_infos[-1]

{'course': '17356L1',
 'session': 'classinsight-cmu_17356L1_weh_5302_201905021030',
 'start_time': Timestamp('2019-05-02 10:30:00'),
 'start_hour': '10:30',
 'classroom': 'weh_5302',
 'completed': True,
 'num_frames': 23999,
 'max_frame': 71994,
 'duration_in_mins': 79}

In [7]:
df_session_info

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins
0,17214A,classinsight-cmu_17214A_dh_2105_201901300930,2019-01-30 09:30:00,09:30,dh_2105,False,0,NaN,NaN
1,17214A,classinsight-cmu_17214A_dh_2105_201902130930,2019-02-13 09:30:00,09:30,dh_2105,False,0,NaN,NaN
2,17214A,classinsight-cmu_17214A_dh_2105_201902200930,2019-02-20 09:30:00,09:30,dh_2105,False,0,NaN,NaN
3,17214A,classinsight-cmu_17214A_dh_2105_201902270930,2019-02-27 09:30:00,09:30,dh_2105,False,0,NaN,NaN
4,17214A,classinsight-cmu_17214A_dh_2105_201903060930,2019-03-06 09:30:00,09:30,dh_2105,False,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
838,17356L1,classinsight-cmu_17356L1_weh_5302_201904231030,2019-04-23 10:30:00,10:30,weh_5302,True,24000,71997.0,79.0
839,17356L1,classinsight-cmu_17356L1_weh_5302_201901241030,2019-01-24 10:30:00,10:30,weh_5302,True,23916,71745.0,79.0
840,17356L1,classinsight-cmu_17356L1_weh_5302_201904251030,2019-04-25 10:30:00,10:30,weh_5302,True,23999,71994.0,79.0
841,17356L1,classinsight-cmu_17356L1_weh_5302_201904301030,2019-04-30 10:30:00,10:30,weh_5302,True,23999,71994.0,79.0


In [8]:
df_session_info = df_session_info[df_session_info.completed]
df_session_info['course_num'] = df_session_info.course.apply(lambda x: int(x[:5]) if len(x)>5 else int(x[:4]))
df_session_info['course_num'].value_counts()

9105     103
21127     84
5410      55
24352     39
6705      37
17356     36
17214     31
5748      28
5391      27
15251     26
5418      26
76270     20
21260     16
82271     16
5681      15
79201     14
57173     13
19603     12
73102     10
36200      9
9214       9
9403       8
5772       8
15424      8
15819      7
15740      7
79388      6
79240      6
6485       6
9217       6
18715      5
82119      5
86375      5
82235      5
21122      5
17346      5
82101      4
19646      3
12411      2
5682       2
17213      1
Name: course_num, dtype: int64

# Select courses and sessions for experiment

In [14]:
# 10 percentile of id count every selected session
def get_percentile_id_count_per_frame(session, percentile_val=10):
    session_id_file = f'/mnt/ci-nas-cache/edulyzeV2/tracking_analysis/session_tracking_info/{session}-front.pb'
    if os.path.exists(session_id_file):
        try:
            return np.percentile(np.nansum(pickle.load(open(session_id_file,'rb')).transpose().values, axis=1), percentile_val)
        except:
            return np.nan
    else:
        return np.nan

In [15]:
df_session_info['10_perc_id_count'] = df_session_info.apply(lambda row: get_percentile_id_count_per_frame(row['session'], 10),axis=1)
df_session_info

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins,course_num,10_perc_id_count
12,17214B,classinsight-cmu_17214B_ph_a21_201901301030,2019-01-30 10:30:00,10:30,ph_a21,True,7476,44850.0,49.0,17214,0.0
13,17214B,classinsight-cmu_17214B_ph_a21_201902271030,2019-02-27 10:30:00,10:30,ph_a21,True,14987,44958.0,49.0,17214,8.0
14,17214B,classinsight-cmu_17214B_ph_a21_201903061030,2019-03-06 10:30:00,10:30,ph_a21,True,14264,42789.0,47.0,17214,8.0
15,17214B,classinsight-cmu_17214B_ph_a21_201903131030,2019-03-13 10:30:00,10:30,ph_a21,True,5472,16413.0,18.0,17214,0.0
16,17214B,classinsight-cmu_17214B_ph_a21_201903201030,2019-03-20 10:30:00,10:30,ph_a21,True,5480,16437.0,18.0,17214,7.0
...,...,...,...,...,...,...,...,...,...,...,...
838,17356L1,classinsight-cmu_17356L1_weh_5302_201904231030,2019-04-23 10:30:00,10:30,weh_5302,True,24000,71997.0,79.0,17356,10.0
839,17356L1,classinsight-cmu_17356L1_weh_5302_201901241030,2019-01-24 10:30:00,10:30,weh_5302,True,23916,71745.0,79.0,17356,17.0
840,17356L1,classinsight-cmu_17356L1_weh_5302_201904251030,2019-04-25 10:30:00,10:30,weh_5302,True,23999,71994.0,79.0,17356,9.0
841,17356L1,classinsight-cmu_17356L1_weh_5302_201904301030,2019-04-30 10:30:00,10:30,weh_5302,True,23999,71994.0,79.0,17356,9.0


In [16]:
df_prefiltered_sessions = df_session_info[df_session_info['10_perc_id_count']>5]
course_session_counts = df_prefiltered_sessions.course.value_counts()
filtered_courses = course_session_counts[course_session_counts>5].index.values
df_prefiltered_sessions = df_prefiltered_sessions[(df_prefiltered_sessions.course.isin(filtered_courses)) & (df_prefiltered_sessions.duration_in_mins>40.)]
df_prefiltered_sessions
                                                  

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins,course_num,10_perc_id_count
13,17214B,classinsight-cmu_17214B_ph_a21_201902271030,2019-02-27 10:30:00,10:30,ph_a21,True,14987,44958.0,49.0,17214,8.0
14,17214B,classinsight-cmu_17214B_ph_a21_201903061030,2019-03-06 10:30:00,10:30,ph_a21,True,14264,42789.0,47.0,17214,8.0
17,17214B,classinsight-cmu_17214B_ph_a21_201904031030,2019-04-03 10:30:00,10:30,ph_a21,True,14998,44991.0,49.0,17214,6.0
18,17214B,classinsight-cmu_17214B_ph_a21_201904101030,2019-04-10 10:30:00,10:30,ph_a21,True,14994,44979.0,49.0,17214,8.0
21,17214B,classinsight-cmu_17214B_ph_a21_201904241030,2019-04-24 10:30:00,10:30,ph_a21,True,14988,44961.0,49.0,17214,8.0
...,...,...,...,...,...,...,...,...,...,...,...
838,17356L1,classinsight-cmu_17356L1_weh_5302_201904231030,2019-04-23 10:30:00,10:30,weh_5302,True,24000,71997.0,79.0,17356,10.0
839,17356L1,classinsight-cmu_17356L1_weh_5302_201901241030,2019-01-24 10:30:00,10:30,weh_5302,True,23916,71745.0,79.0,17356,17.0
840,17356L1,classinsight-cmu_17356L1_weh_5302_201904251030,2019-04-25 10:30:00,10:30,weh_5302,True,23999,71994.0,79.0,17356,9.0
841,17356L1,classinsight-cmu_17356L1_weh_5302_201904301030,2019-04-30 10:30:00,10:30,weh_5302,True,23999,71994.0,79.0,17356,9.0


In [33]:
df_sessions_new = df_prefiltered_sessions[df_prefiltered_sessions.classroom.isin(['ghc_4211','ph_a21','ph_225b']) & (df_prefiltered_sessions.course.isin(['17214B','17214C','05410B']))].reset_index(drop=True)

In [35]:
df_sessions_new.session.values.tolist()

['classinsight-cmu_17214B_ph_a21_201902271030',
 'classinsight-cmu_17214B_ph_a21_201903061030',
 'classinsight-cmu_17214B_ph_a21_201904031030',
 'classinsight-cmu_17214B_ph_a21_201904101030',
 'classinsight-cmu_17214B_ph_a21_201904241030',
 'classinsight-cmu_17214C_ph_225b_201904031130',
 'classinsight-cmu_17214C_ph_225b_201904101130',
 'classinsight-cmu_17214C_ph_225b_201904171130',
 'classinsight-cmu_17214C_ph_225b_201904241130',
 'classinsight-cmu_17214C_ph_225b_201905011130',
 'classinsight-cmu_05410B_ghc_4211_201902111500',
 'classinsight-cmu_05410B_ghc_4211_201903181500',
 'classinsight-cmu_05410B_ghc_4211_201904081500',
 'classinsight-cmu_05410B_ghc_4211_201904151500',
 'classinsight-cmu_05410B_ghc_4211_201904221500',
 'classinsight-cmu_05410B_ghc_4211_201901281500']

In [34]:
df_sessions_new.to_csv('../cache/evaluation_tracking/selected_sessions_ve_jul17.csv',index=False)

In [20]:
df_prefiltered_sessions.classroom.unique()

array(['ph_a21', 'ph_225b', '407sc_104', 'ghc_4301', 'weh_5304',
       'weh_6423', 'ghc_4102', 'ph_a22', 'ghc_4211', 'weh_5302',
       'ghc_4101', 'ghc_5222', 'ph_a18c'], dtype=object)

In [49]:
# randomly select 10 courses.
np.random.seed(43)
selected_courses = df_prefiltered_sessions.course.drop_duplicates().sample(n=9)
print(f"Selected courses:{selected_courses}")

# select 5 sessions per course 
df_selected_sessions = pd.DataFrame()
for course in selected_courses.values:
    if df_prefiltered_sessions[df_prefiltered_sessions.course==course].shape[0]>5:
        df_selected_sessions = pd.concat([df_selected_sessions,df_prefiltered_sessions[df_prefiltered_sessions.course==course].sample(n=5)])
df_selected_sessions

Selected courses:723    09105D
610    05681A
582    05418A
624    05748A
123    21127J
689    09105C
96     17214E
557    05410B
66     05410A
Name: course, dtype: object


,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins,course_num,10_perc_id_count
623,05681A,classinsight-cmu_05681A_ghc_4301_201905011630,2019-05-01 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,20.0
620,05681A,classinsight-cmu_05681A_ghc_4301_201904171630,2019-04-17 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,22.0
612,05681A,classinsight-cmu_05681A_ghc_4301_201902201630,2019-02-20 16:30:00,16:30,ghc_4301,True,32962,98883.0,109.0,5681,24.0
619,05681A,classinsight-cmu_05681A_ghc_4301_201904101630,2019-04-10 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,24.0
621,05681A,classinsight-cmu_05681A_ghc_4301_201901231630,2019-01-23 16:30:00,16:30,ghc_4301,True,32988,98961.0,109.0,5681,25.0
599,05418A,classinsight-cmu_05418A_ghc_4102_201902251200,2019-02-25 12:00:00,12:00,ghc_4102,True,23950,71847.0,79.0,5418,23.0
583,05418A,classinsight-cmu_05418A_ghc_4102_201904081200,2019-04-08 12:00:00,12:00,ghc_4102,True,23999,71994.0,79.0,5418,16.0
591,05418A,classinsight-cmu_05418A_ghc_4102_201905011200,2019-05-01 12:00:00,12:00,ghc_4102,True,23998,71991.0,79.0,5418,19.0
590,05418A,classinsight-cmu_05418A_ghc_4102_201904291200,2019-04-29 12:00:00,12:00,ghc_4102,True,23999,71994.0,79.0,5418,17.0
588,05418A,classinsight-cmu_05418A_ghc_4102_201904011200,2019-04-01 12:00:00,12:00,ghc_4102,True,23998,71991.0,79.0,5418,15.0


In [67]:
df_selected_sessions.session.values.tolist()

['classinsight-cmu_05681A_ghc_4301_201905011630',
 'classinsight-cmu_05681A_ghc_4301_201904171630',
 'classinsight-cmu_05681A_ghc_4301_201902201630',
 'classinsight-cmu_05681A_ghc_4301_201904101630',
 'classinsight-cmu_05681A_ghc_4301_201901231630',
 'classinsight-cmu_05418A_ghc_4102_201902251200',
 'classinsight-cmu_05418A_ghc_4102_201904081200',
 'classinsight-cmu_05418A_ghc_4102_201905011200',
 'classinsight-cmu_05418A_ghc_4102_201904291200',
 'classinsight-cmu_05418A_ghc_4102_201904011200',
 'classinsight-cmu_05748A_ghc_4101_201902141630',
 'classinsight-cmu_05748A_ghc_4101_201904021630',
 'classinsight-cmu_05748A_ghc_4101_201902051630',
 'classinsight-cmu_05748A_ghc_4101_201902281630',
 'classinsight-cmu_05748A_ghc_4101_201903071630',
 'classinsight-cmu_21127J_ghc_4102_201904230930',
 'classinsight-cmu_21127J_ghc_4102_201903260930',
 'classinsight-cmu_21127J_ghc_4102_201904160930',
 'classinsight-cmu_21127J_ghc_4102_201904300930',
 'classinsight-cmu_21127J_ghc_4102_201903190930',


In [51]:
# df_selected_sessions.to_csv('../cache/evaluation_tracking/selected_sessions.csv',index=False)

In [62]:
# select 5 snippets of 1 min from 5 different parts of lecture.
np.random.seed(43)
snippets = []
num_snippets = 5
for row_idx, row in df_selected_sessions.iterrows():
    session_splits = np.array_split(np.arange(int(row['duration_in_mins'])), num_snippets)
    selected_mins = [np.random.choice(xr) for xr in session_splits]
    for xr in selected_mins:
        row_dict = row.to_dict()
        row_dict.update({'snippet_min':xr})
        snippets.append(row_dict)
        # print(row_dict)
    # selected_snippet_frames = [np.arange(xr*15*60, (xr+1)*15*60) for xr in selected_mins]
df_session_snippets = pd.DataFrame(snippets)
df_session_snippets

,course,session,start_time,start_hour,classroom,completed,num_frames,max_frame,duration_in_mins,course_num,10_perc_id_count,snippet_min
0,05681A,classinsight-cmu_05681A_ghc_4301_201905011630,2019-05-01 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,20.0,4
1,05681A,classinsight-cmu_05681A_ghc_4301_201905011630,2019-05-01 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,20.0,22
2,05681A,classinsight-cmu_05681A_ghc_4301_201905011630,2019-05-01 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,20.0,61
3,05681A,classinsight-cmu_05681A_ghc_4301_201905011630,2019-05-01 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,20.0,87
4,05681A,classinsight-cmu_05681A_ghc_4301_201905011630,2019-05-01 16:30:00,16:30,ghc_4301,True,32998,98991.0,109.0,5681,20.0,104
...,...,...,...,...,...,...,...,...,...,...,...,...
120,05410A,classinsight-cmu_05410A_ghc_4301_201902181500,2019-02-18 15:00:00,15:00,ghc_4301,True,23843,71526.0,79.0,5410,17.0,7
121,05410A,classinsight-cmu_05410A_ghc_4301_201902181500,2019-02-18 15:00:00,15:00,ghc_4301,True,23843,71526.0,79.0,5410,17.0,24
122,05410A,classinsight-cmu_05410A_ghc_4301_201902181500,2019-02-18 15:00:00,15:00,ghc_4301,True,23843,71526.0,79.0,5410,17.0,43
123,05410A,classinsight-cmu_05410A_ghc_4301_201902181500,2019-02-18 15:00:00,15:00,ghc_4301,True,23843,71526.0,79.0,5410,17.0,56


In [63]:
len(snippets)

125

In [64]:
df_session_snippets.to_csv('../cache/evaluation_tracking/selected_session_snippets.csv',index=False)